In [19]:
# -*- coding: cp1252 -*-


from mechanize import Browser
import os
import pandas as pd
import html5lib
from bs4 import BeautifulSoup  


#os.chdir(r"E:\Dropbox\Dropbox\0_DB\A_Elaborazioni\Coeweb")

class coeweb(object):
    
    def __init__(self, link):
        self.pandas_db = pd.DataFrame()
        self.link = link
        br = Browser()
        br.open(self.link)
        br.select_form(nr=0)
        
        self.paesi = []
        self.paesi_descizione = []
        control = br.form.find_control("PAESE")
        if control.type == "select":  
            for item in control.items:
                self.paesi.append(item.name)
                self.paesi_descizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.territori = []
        self.territori_descrizione = []
        control = br.form.find_control("CODTERRITORIO")
        if control.type == "select":  
            for item in control.items:
                self.territori.append((item.name, str([label.text  for label in item.get_labels()])))
                self.territori_descrizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.anni = []
        control = br.form.find_control("ANNO")
        if control.type == "select":  
            for item in control.items:
                self.anni.append((item.name, str([label.text  for label in item.get_labels()])))
        self.mese = []
        control = br.form.find_control("MESE")
        if control.type == "select": 
            for item in control.items:
                self.mese.append((item.name, str([label.text  for label in item.get_labels()])))
        self.cumulato = []
        control = br.form.find_control("TIPOVALORE")
        for item in control.items:
            self.cumulato.append((item.name, str([label.text  for label in item.get_labels()])))
        
        self.bric = ['508', '720', '664', '75']
        #[('508', "['508 - Brasile']"), ('720', "['720 - Cina']"), ('664', "['664 - India']"), ('75', "['75 - Russia']")]
    
    def convert_string_euro(self,x):
        '''questa funzione rimuove la formattazione dei numeri di coeweb, trasformando il testo in numeri'''
        try: 
            return float(x.replace(".", ""))
        except:
            pass
    

    def scarica_uno(self, territorio = ["309048"], paese = ["1033"], cumulato = ['M'], anno = ["2013"], mese = ["4"]  ):# , paese, anno, trimestre, modalita)
        
        br = Browser()
        br.open(self.link)

        br.select_form(nr=0)

        control_territorio = br.form.find_control("CODTERRITORIO")
        br[control_territorio.name] = territorio #["309048"]

        control_paese = br.form.find_control("PAESE")
        br[control_paese.name] = paese #["309048"]

        control_anno = br.form.find_control("ANNO")
        br[control_anno.name] = anno #["309048"]

        control_mese = br.form.find_control("MESE")
        br[control_mese.name] = mese #["309048"]

        control_cumulato = br.form.find_control("TIPOVALORE")
        br[control_cumulato.name] = cumulato #["309048"]

        nome_stringa = territorio[0] + "_" + paese[0] + "_" + anno[0] + "_" + mese[0] + "_" + cumulato[0]


        response = br.submit()
        out_file = open(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), "w")
        out_file.write(response.read()) 
        out_file.close()
        #os.system("start " + nome_stringa +".html")
        self.dati = pd.read_html(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), header= 0, index_col=0, encoding = "cp1252")#'utf8'
        #os.remove(territorio[0] +".html")
        print "download_" + nome_stringa
        br.close()
        return self.dati
        
        # i dati dovrebbero essere raccolti in un pandas 
        
    def scarica_molti(self, territorio = ["309048"],   lista_paesi = None, lista_anni = None, lista_mesi = None, lista_cumulato = ["M"]):
        ''' questa funzione permette di scaricare per uno o più territori uno o più settori, per uno o più periodi di tempo'''
        #self.pandas_db = pd.DataFrame()
        
        def nome_paese(dati, codice):
            for item in dati:
                if item[0] == codice:
                    pulito1= item[1].split("'",)[1]
                    pulito2 = pulito1.split(" - ",)[1]
                    print pulito2
                    return pulito2
        
        def nome_territorio(dati, codice):
            for item in dati:
                if item[0] == codice:
                    try:
                        pulito1= item[1].split("'")[1]
                        pulito1 = pulito1.replace("-","")
                        pulito1 = pulito1.replace("'","")
                        return pulito1
                    except:
                        pass
        
        
        for item_territori in territorio:
            for item_paesi in lista_paesi:
                for item_anni in lista_anni:
                    for item_mesi in lista_mesi:

                        dataset =  self.scarica_uno([item_territori], paese = [str(item_paesi)],  anno = [str(item_anni)], mese = [str(item_mesi)], cumulato = lista_cumulato)
                        dataset = dataset[1]#.set_index(territorio, str(item_paesi), str(item_anni), str(item_mesi)])
                        
                        
                        dataset["territorio"] = nome_territorio(self.territori_descrizione,  item_territori)  
                        #dataset["territorio"] = item_territori
                        
                        #nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["paese"] = nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["anno"] = str(item_anni)
                        dataset["mese"] = str(item_mesi)
                        dataset["cumulato"] = lista_cumulato[0]
                        dataset.set_index(["territorio","paese","anno","mese","cumulato"], append = True, inplace = True)
                        dataset["DATI EXPORT"] = dataset["EXP" + str(item_anni)]
                          
                        
                        
                        dataset = dataset["DATI EXPORT"].apply(lambda x: self.convert_string_euro(x))
                        
                        
                        
                        # datasetcompleto
                        self.pandas_db = pd.concat([self.pandas_db, pd.DataFrame(dataset)])
                        # settori ateco disponibili
                        self.pandas_db.settori = set(self.pandas_db.reset_index()["Divisioni"])
                        
                            #print "errore_" + territorio[0] + "_" + str(item_paesi) + "_" + str(item_anni) + "_" + str(item_mesi) + "_" + lista_cumulato[0]

        for scaricato in os.listdir(os.path.join(os.getcwd(), "scaricati")):
            os.remove(os.path.join(os.getcwd(), "scaricati", scaricato))
        

        print "eseguito"
    
    
    
    def analizza_settori(self, settori, interpolazione = "no"):
        ''' questa funzione restituisce un dataframe e un dizionario contenente le serie storiche per i settori inseriti come argomento [lista]. 
        Effettua anche l'interpolazione dei dati mancanti'''
        self.dict_settore = {}
        for item in settori:
            dati_settore = self.pandas_db.xs(item, level=0)
            if interpolazione == "yes":
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
            else:
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x))
            self.dict_settore[item] = dati_settore["DATI EXPORT"]
        self.dataframe_settori = pd.DataFrame(self.dict_settore)
        return self.dataframe_settori

In [20]:
def convert_string_euro(x):
    try: 
        return float(x.replace(".", ""))
    except:
        pass

In [21]:
os.getcwd()

'e:\\Dropbox\\Dropbox\\0_DB\\coeweb'

In [22]:

lista_anni = [str(i) for i in range(2009,2016)]


In [23]:
Firenze = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [24]:
metropolitana = [('309', "['---Toscana']"),
 ('309047', "['------Pistoia']"),
 ('309048', "['------Firenze']"),
 ('309100', "['------Prato']")]

In [25]:
CodiciMetropolitana = []
for item in metropolitana:
    CodiciMetropolitana.append(item[0])

In [26]:
Firenze.scarica_molti(territorio = CodiciMetropolitana, lista_paesi = ['6'], lista_anni = lista_anni, lista_mesi = ["9"], lista_cumulato = ['C'])

download_309_6_2009_9_C
Regno Unito
download_309_6_2010_9_C
Regno Unito
download_309_6_2011_9_C
Regno Unito
download_309_6_2012_9_C
Regno Unito
download_309_6_2013_9_C
Regno Unito
download_309_6_2014_9_C
Regno Unito
download_309_6_2015_9_C
Regno Unito
download_309047_6_2009_9_C
Regno Unito
download_309047_6_2010_9_C
Regno Unito
download_309047_6_2011_9_C
Regno Unito
download_309047_6_2012_9_C
Regno Unito
download_309047_6_2013_9_C
Regno Unito
download_309047_6_2014_9_C
Regno Unito
download_309047_6_2015_9_C
Regno Unito
download_309048_6_2009_9_C
Regno Unito
download_309048_6_2010_9_C
Regno Unito
download_309048_6_2011_9_C
Regno Unito
download_309048_6_2012_9_C
Regno Unito
download_309048_6_2013_9_C
Regno Unito
download_309048_6_2014_9_C
Regno Unito
download_309048_6_2015_9_C
Regno Unito
download_309100_6_2009_9_C
Regno Unito
download_309100_6_2010_9_C
Regno Unito
download_309100_6_2011_9_C
Regno Unito
download_309100_6_2012_9_C
Regno Unito
download_309100_6_2013_9_C
Regno Unito
downloa

In [27]:
dati_toscana_completi = Firenze.pandas_db

In [71]:
dati_toscana_completi = dati_toscana_completi.reset_index()

In [86]:
dati_toscana_completi["anno"] = pd.to_datetime(dati_toscana_completi.anno, format="%Y")

In [87]:
dati_toscana_completi.drop(["paese", "mese", "cumulato"], axis = 1, inplace = True)

ValueError: labels ['paese' 'mese' 'cumulato'] not contained in axis

0     2009-01-01
1     2009-01-01
2     2009-01-01
3     2009-01-01
4     2009-01-01
5     2009-01-01
6     2009-01-01
7     2009-01-01
8     2009-01-01
9     2009-01-01
10    2009-01-01
11    2009-01-01
12    2009-01-01
13    2009-01-01
14    2009-01-01
15    2009-01-01
16    2009-01-01
17    2009-01-01
18    2009-01-01
19    2009-01-01
20    2009-01-01
21    2009-01-01
22    2009-01-01
23    2009-01-01
24    2009-01-01
25    2009-01-01
26    2009-01-01
27    2009-01-01
28    2009-01-01
29    2009-01-01
         ...    
941   2015-01-01
942   2015-01-01
943   2015-01-01
944   2015-01-01
945   2015-01-01
946   2015-01-01
947   2015-01-01
948   2015-01-01
949   2015-01-01
950   2015-01-01
951   2015-01-01
952   2015-01-01
953   2015-01-01
954   2015-01-01
955   2015-01-01
956   2015-01-01
957   2015-01-01
958   2015-01-01
959   2015-01-01
960   2015-01-01
961   2015-01-01
962   2015-01-01
963   2015-01-01
964   2015-01-01
965   2015-01-01
966   2015-01-01
967   2015-01-01
968   2015-01-

In [88]:
dati_toscana_completi.to_excel("dati_metropolitana_completi.xlsx")

In [29]:
Firenze

In [34]:
serie_storica = dati_toscana_completi[dati_toscana_completi.index.get_level_values('Divisioni').isin(['Totale'])].unstack("anno")

In [41]:
#["2015"]

Divisioni  territorio  paese        mese  cumulato
Totale     Firenze     Regno Unito  9     C           5.816399e+08
           Pistoia     Regno Unito  9     C           1.242330e+08
           Prato       Regno Unito  9     C           1.427064e+08
           Toscana     Regno Unito  9     C           1.802210e+09
Name: 2015, dtype: float64

In [58]:
%matplotlib inline

serie_storica["DATI EXPORT"].divide(serie_storica["DATI EXPORT"]["2009"], axis = 0) * 100

anno                                             2009        2010        2011  \
Divisioni territorio paese       mese cumulato                                  
Totale    Firenze    Regno Unito 9    C         100.0  124.888999  115.685097   
          Pistoia    Regno Unito 9    C         100.0  117.163396  122.527492   
          Prato      Regno Unito 9    C         100.0  139.286136  132.357143   
          Toscana    Regno Unito 9    C         100.0   96.364468   99.402493   

anno                                                  2012        2013  \
Divisioni territorio paese       mese cumulato                           
Totale    Firenze    Regno Unito 9    C         124.545145  176.578336   
          Pistoia    Regno Unito 9    C         114.321328  123.339313   
          Prato      Regno Unito 9    C         110.313583  114.053888   
          Toscana    Regno Unito 9    C          97.228852  105.662722   

anno                                                  2014        2015  
Divisioni territorio paese       mese cumulato                          
Totale    Firenze    Regno Unito 9    C         184.469509  200.457805  
          Pistoia    Regno Unito 9    C         129.887862  145.614288  
          Prato      Regno Unito 9    C         135.769640  173.755019  
          Toscana    Regno Unito 9    C         110.345702  117.888034